In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163761 sha256=7e89ea47f913403708f9513449a14302c8cda38d8110f08846c266c7e13d89fb
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import SVD
from surprise.model_selection import cross_validate

In [ ]:
from google.colab import drive
from google.colab import files

# Google Drive Mount
drive.mount('/content/drive')

# Local File Upload
uploaded = files.upload()

# Move the file
for filename in uploaded.keys():
    # Move the uploaded file to the desired location in Google Drive.
    !mv "{filename}" "/content/drive/MyDrive/{filename}"

Mounted at /content/drive


Saving food.csv to food.csv


In [ ]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/food.csv')

In [ ]:
df.head()

,name,minutes,contributor_id,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,recipe_id,date,rating,review
0,whatever floats your boat brownies,35,37305,14,"['preheat oven to 350f', 'grease an 8 inch squ...","these are absolutely the chewiest, moistest, f...","['butter', 'unsweetened cocoa', 'sugar', 'eggs...",14.0,390.7,30.0,161.0,7.0,12.0,50.0,17.0,2226227,32204,2012-12-21,5,So yummy with 1.5 cups of m&ms and .5 cup shre...
1,whatever floats your boat brownies,35,37305,14,"['preheat oven to 350f', 'grease an 8 inch squ...","these are absolutely the chewiest, moistest, f...","['butter', 'unsweetened cocoa', 'sugar', 'eggs...",14.0,390.7,30.0,161.0,7.0,12.0,50.0,17.0,45980,32204,2002-06-24,5,not hard to make and delious. They never last ...
2,whatever floats your boat brownies,35,37305,14,"['preheat oven to 350f', 'grease an 8 inch squ...","these are absolutely the chewiest, moistest, f...","['butter', 'unsweetened cocoa', 'sugar', 'eggs...",14.0,390.7,30.0,161.0,7.0,12.0,50.0,17.0,61493,32204,2003-03-03,5,"Delicious! Don't change a thing, except you m..."
3,whatever floats your boat brownies,35,37305,14,"['preheat oven to 350f', 'grease an 8 inch squ...","these are absolutely the chewiest, moistest, f...","['butter', 'unsweetened cocoa', 'sugar', 'eggs...",14.0,390.7,30.0,161.0,7.0,12.0,50.0,17.0,104492,32204,2003-09-17,5,These brownies were decadent! There are no ot...
4,whatever floats your boat brownies,35,37305,14,"['preheat oven to 350f', 'grease an 8 inch squ...","these are absolutely the chewiest, moistest, f...","['butter', 'unsweetened cocoa', 'sugar', 'eggs...",14.0,390.7,30.0,161.0,7.0,12.0,50.0,17.0,68715,32204,2003-10-23,5,Oh my MY MY!!!THESE ARE GOOD!!!! Soooo chocola...


In [ ]:
# find the minimum and maximum values in the 'protein (PDV)'
min_protein = df['protein (PDV)'].min()
max_protein = df['protein (PDV)'].max()

print("Minimum protein:", min_protein)
print("Maximum protein:", max_protein)

Minimum protein: 2.0
Maximum protein: 184.5


In [ ]:
# 원래 데이터의 최소값과 최대값 설정
original_min = 2
original_max = 184.5

# 새로운 최소값과 최대값
new_min = 1
new_max = 5

# 'protein (PDV)' 컬럼을 정규화
df['protein_normalized'] = new_min + ((df['protein (PDV)'] - original_min) * (new_max - new_min) / (original_max - original_min))

In [ ]:
df['protein_normalized']

0        1.219178
1        1.219178
2        1.219178
3        1.219178
4        1.219178
           ...   
49929    1.065753
49930    1.065753
49931    1.065753
49932    1.065753
49933    1.065753
Name: protein_normalized, Length: 49934, dtype: float64

In [ ]:
# Define a Reader
reader = Reader(line_format='user item rating', rating_scale=(min_protein, max_protein))

# Data load
data = Dataset.load_from_df(df[['user_id', 'recipe_id', 'protein_normalized']], reader)

In [ ]:
# Create an item-based KNNBasic model
sim_options = {
    'name': 'cosine',  # Similarity measurement criteria
    'user_based': False  # Using item-based filtering
}

In [ ]:
# split data
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
knn_model = KNNBasic(sim_options=sim_options)

In [ ]:
# train model
knn_model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
def get_recommendations(user_id, calorie_limit):
    # Retrieve all unique recipe_ids
    all_recipe_ids = df['recipe_id'].unique()

    # Get recipe_ids already rated by the user
    rated_recipe_ids = df.loc[df['user_id'] == user_id, 'recipe_id'].tolist()

    # Find unrated recipes
    unrated_recipe_ids = [recipe_id for recipe_id in all_recipe_ids if recipe_id not in rated_recipe_ids]

    # Generate predictions for all unrated recipes
    recommendations = []
    for recipe_id in unrated_recipe_ids:
        prediction = knn_model.predict(user_id, recipe_id)
        recommendations.append((recipe_id, prediction.est))

    # Sort them in descending order based on predicted ratings
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:10]

    # Filter recommendations based on calorie limit and unpack correctly
    filtered_recommendations = []
    for recipe_id, estimated_rating in top_recommendations:
        calories = df.loc[df['recipe_id'] == recipe_id, 'calories'].iloc[0]
        if calories <= calorie_limit:
            recipe_info = df.loc[df['recipe_id'] == recipe_id, ['name', 'description']].iloc[0]
            filtered_recommendations.append((recipe_id, recipe_info['name'], recipe_info['description']))

    return pd.DataFrame(filtered_recommendations, columns=['recipe_id', 'name', 'description'])

In [ ]:
# Example
user_id = 29956
calorie_limit = 2000 #standard based on the korean nutrition management table https://www.gne.go.kr/upload_data/board_data/BBS_0000027/164930888639904.pdf
recommendations = get_recommendations(user_id, calorie_limit)
recommendations

,recipe_id,name,description
0,32204,whatever floats your boat brownies,"these are absolutely the chewiest, moistest, f..."
1,132263,5 minute vegan pancakes,"a cinch to make and tastes wonderful, if you l..."
2,28768,absolute best ever lasagna,my family's favorite. this is a very meaty las...
3,92095,authentic italian meatballs,meatball recipes are often challenged by chefs...
4,25885,banana banana bread,why compromise the banana flavor? my banana br...
5,2886,best banana bread,you'll never need another banana bread recipe ...
6,67256,best ever banana cake with cream cheese frosting,this is one of (if not) the best banana cake i...
7,27210,best grilled pork chops,this is a simple recipe i have adapted from co...
8,107786,beth s melt in your mouth barbecue ribs oven,i almost didn't want to share this ribs recipe...
9,141983,better than olive garden alfredo sauce,this is the best alfredo sauce ever. this is s...


In [ ]:
mf_model = SVD()
mf_model.fit(trainset)

In [ ]:
def get_recommendations(user_id, calorie_limit):
    # Retrieve all unique recipe_ids
    all_recipe_ids = df['recipe_id'].unique()

    # Get recipe_ids already rated by the user
    rated_recipe_ids = df.loc[df['user_id'] == user_id, 'recipe_id'].tolist()

    # Find unrated recipes
    unrated_recipe_ids = [recipe_id for recipe_id in all_recipe_ids if recipe_id not in rated_recipe_ids]

    # Generate predictions for all unrated recipes
    recommendations = []
    for recipe_id in unrated_recipe_ids:
        prediction = mf_model.predict(user_id, recipe_id)
        recommendations.append((recipe_id, prediction.est))

    # Sort them in descending order based on predicted ratings
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:10]

    # Filter recommendations based on calorie limit and unpack correctly
    filtered_recommendations = []
    for recipe_id, estimated_rating in top_recommendations:
        calories = df.loc[df['recipe_id'] == recipe_id, 'calories'].iloc[0]
        if calories <= calorie_limit:
            recipe_info = df.loc[df['recipe_id'] == recipe_id, ['name', 'description']].iloc[0]
            filtered_recommendations.append((recipe_id, recipe_info['name'], recipe_info['description']))

    return pd.DataFrame(filtered_recommendations, columns=['recipe_id', 'name', 'description'])

In [ ]:
# Example
user_id = 29956
calorie_limit = 2000
recommendations = get_recommendations(user_id, calorie_limit)
recommendations

,recipe_id,name,description
0,69173,kittencal s italian melt in your mouth meatballs,cooking the meatballs in simmering pasta sauce...
1,107786,beth s melt in your mouth barbecue ribs oven,i almost didn't want to share this ribs recipe...
2,8701,should be illegal oven bbq ribs,after not being able to find restaurant ribs t...
3,66121,p f chang s mongolian beef by todd wilbur,update: this recipe was created by todd wilbur...
4,27210,best grilled pork chops,this is a simple recipe i have adapted from co...
5,53914,mama s supper club tilapia parmesan,you asked for it. here it is. thanks so much t...
6,27208,to die for crock pot roast,"amazing flavor, and so simple! no salt needed ..."
7,76930,funky chicken with sesame noodles,this has to be one of the most amazing chicken...
8,73166,the best chili you will ever taste,this is the best chili recipe i have ever trie...
9,89204,crock pot chicken with black beans cream cheese,i love this crock-pot chicken recipe for two r...


In [ ]:
# KNNBasic model metric
knn_results = cross_validate(knn_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# SVD model metril
mf_results = cross_validate(mf_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# result
print("KNNBasic 모델의 평균 RMSE: {:.4f}".format(knn_results['test_rmse'].mean()))
print("KNNBasic 모델의 평균 MAE: {:.4f}".format(knn_results['test_mae'].mean()))

print("\nMF 모델의 평균 RMSE: {:.4f}".format(mf_results['test_rmse'].mean()))
print("MF 모델의 평균 MAE: {:.4f}".format(mf_results['test_mae'].mean()))

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9434  0.9539  0.9487  0.9403  0.9547  0.9482  0.0057  
MAE (testset)     0.7640  0.7745  0.7704  0.7634  0.7745  0.7694  0.0049  
Fit time          0.01    0.03    0.03    0.03    0.03    0.03    0.01    
Test time         0.10    0.12    0.21    0.11    0.11    0.13    0.04    
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5665  0.5697  0.5632  0.5644  0.5722 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
# predict the test data
predictions = mf_model.test(testset)

# Convert Predicted Ratings to 0 or 1 for Binary Classification
threshold = 3.5
binary_predictions = [1 if pred.est >= threshold else 0 for pred in predictions]

# Convert Actual Ratings to Binary Classification
actual_ratings = [1 if actual_rating >= threshold else 0 for _, _, actual_rating in testset]

# Precision, Recall, F1-score calculate
precision = precision_score(actual_ratings, binary_predictions)
recall = recall_score(actual_ratings, binary_predictions)
f1 = f1_score(actual_ratings, binary_predictions)

# result
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))

Precision: 1.0000
Recall: 1.0000
F1-score: 1.0000


In [ ]:
df = df.drop(df[df['rating'] == 0].index)

In [ ]:
# 'rating' 열의 최솟값과 최댓값을 찾습니다.
min_rating = df['rating'].min()
max_rating = df['rating'].max()

# Define a Reader object
reader = Reader(rating_scale=(min_rating, max_rating))

# Data load
r_data = Dataset.load_from_df(df[['user_id', 'recipe_id', 'rating']], reader)

In [ ]:
# Split the data
r_trainset, r_testset = train_test_split(r_data, test_size=0.2, random_state=42)

In [ ]:
# train model
mf_model.fit(r_trainset)
rmf_results = cross_validate(mf_model, r_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# result
print("MF 모델의 평균 RMSE: {:.4f}".format(rmf_results['test_rmse'].mean()))
print("MF 모델의 평균 MAE: {:.4f}".format(rmf_results['test_mae'].mean()))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7190  0.7512  0.7290  0.7347  0.7449  0.7358  0.0114  
MAE (testset)     0.4516  0.4600  0.4516  0.4580  0.4597  0.4562  0.0038  
Fit time          0.82    0.77    0.82    0.80    0.81    0.80    0.02    
Test time         0.06    0.06    0.06    0.07    0.05    0.06    0.01    
MF 모델의 평균 RMSE: 0.7358
MF 모델의 평균 MAE: 0.4562


In [ ]:
# predict the r_test data
predictions = mf_model.test(r_testset)

# Convert Predicted Ratings to 0 or 1 for Binary Classification
threshold = 3.5
binary_predictions = [1 if pred.est >= threshold else 0 for pred in predictions]

# Convert Actual Ratings to Binary Classification
actual_ratings = [1 if actual_rating >= threshold else 0 for _, _, actual_rating in r_testset]

# Precision, Recall, F1-score calculate
precision = precision_score(actual_ratings, binary_predictions)
recall = recall_score(actual_ratings, binary_predictions)
f1 = f1_score(actual_ratings, binary_predictions)

# result
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))

Precision: 0.9415
Recall: 0.9995
F1-score: 0.9696
